# Y2017M11D29_RH_totalWW_totalWN_WS_Pixel_EE_V01

* Purpose of script: calculate total demand WW WN and water stress at pixel level
* Kernel used: python27
* Date created: 20171129 

In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

('Y2017M11D29', 'UTC 16:22')


'2.7.13 |Continuum Analytics, Inc.| (default, Dec 20 2016, 23:09:15) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
EE_PATH = "projects/WRI-Aquaduct/PCRGlobWB20V07"

SCRIPT_NAME = "Y2017M11D29_RH_totalWW_totalWN_WS_Pixel_EE_V01"

OUTPUT_VERSION = 1

DIMENSION5MIN = {}
DIMENSION5MIN["x"] = 4320
DIMENSION5MIN["y"] = 2160
CRS = "EPSG:4326"

MAXPIXELS =1e10

YEARMIN = 1960
YEARMAX = 2014

In [3]:
import ee
import subprocess

In [4]:
ee.Initialize()

In [5]:
sectors = ["PDom","PInd","PIrr","PLiv"]
demandTypes = ["WW","WN"]
temporalResolutions = ["year","month"]

In [6]:
dimensions = "%sx%s" %(DIMENSION5MIN["x"],DIMENSION5MIN["y"])

In [7]:
crsTransform = [
                0.0833333309780367,
                0,
                -179.99999491255934,
                0,
                -0.0833333309780367,
                90.00000254430942
              ]

In [18]:
def createCollections(demandType,temporalResolution):
    icId = "global_historical_PTot%s_%s_millionm3_5min_1960_2014" %(demandType,temporalResolution)
    command = "earthengine create collection %s/%s" %(EE_PATH,icId) 
    result = subprocess.check_output(command,shell=True)
    print(command,result)

def createCollectionsWS(temporalResolution):
    icId = "global_historical_WS5min_%s_millionm3_5min_1960_2014" %(temporalResolution)
    command = "earthengine create collection %s/%s" %(EE_PATH,icId) 
    result = subprocess.check_output(command,shell=True)
    print(command,result)    

def totalDemand(year,month,demandType,temporalResolution):
    elapsed = datetime.datetime.now() - startLoop
    print(year,month,demandType,temporalResolution)
    print(elapsed)
    d = {}
    keys = []
    properties = {"indicator":"PTot%s" %(demandType) ,
                  "temporal_range_max": 2014,
                  "ingested_by":"RutgerHofste",
                  "units":"millionm3",
                  "temporal_resolution":temporalResolution,
                  "exportdescription":"PTot%s_%sY%0.4dM%0.2d" %(demandType,temporalResolution,year,month),
                  "temporal_range_min":1960,
                  "month": month,
                  "year": year,
                  "demandType":demandType,
                  "script_used": SCRIPT_NAME,
                  "version": OUTPUT_VERSION
                 }
    
    
    for sector in sectors:
        
        key = "%s%s" %(sector,demandType)
        keys.append(key)
        ic = ee.ImageCollection("%s/global_historical_%s%s_%s_millionm3_5min_1960_2014" %(EE_PATH,sector,demandType,temporalResolution))
        
        if temporalResolution == "year":
            image = ee.Image(ic.filter(ee.Filter.eq("year",year)).first())
        elif temporalResolution == "month":
            image = ee.Image(ic.filter(ee.Filter.eq("year",year)).filter(ee.Filter.eq("month",month)).first())
        else:
            image = -9999
        d[key] = image
    
    
    totalImage = ee.Image(d[keys[0]].add(d[keys[1]]).add(d[keys[2]]).add(d[keys[3]]))
    totalImage = totalImage.set(properties)
    
    description = "PTot%s_%sY%0.4dM%0.2dV%0.2d" %(demandType,temporalResolution,year,month,OUTPUT_VERSION)
    assetID = "%s/global_historical_PTot%s_%s_millionm3_5min_1960_2014/global_historical_PTot%s_%s_millionm3_5min_1960_2014Y%0.4dM%0.2d" %(EE_PATH,demandType,temporalResolution,demandType,temporalResolution,year,month)
    
    task = ee.batch.Export.image.toAsset(
        image =  ee.Image(totalImage),
        description = description,
        assetId = assetID,
        dimensions = dimensions,
        crs = CRS,
        crsTransform = crsTransform,
        maxPixels = MAXPIXELS     
    )
    task.start() 
    
    
def waterStressUncapped(year,month,temporalResolution):
    elapsed = datetime.datetime.now() - startLoop
    print(year,month,temporalResolution)
    print(elapsed)
    d = {}
    keys = []
    properties = {"indicator":"WS5min" ,
                  "temporal_range_max": 2014,
                  "ingested_by":"RutgerHofste",
                  "units":"dimensionless",
                  "temporal_resolution":temporalResolution,
                  "exportdescription":"WS5min_%sY%0.4dM%0.2d" %(temporalResolution,year,month),
                  "temporal_range_min":1960,
                  "month": month,
                  "year": year,
                  "script_used": SCRIPT_NAME,
                  "version": OUTPUT_VERSION
                 }
    
    icWW = ee.ImageCollection("%s/global_historical_PTotWW_%s_millionm3_5min_1960_2014" %(EE_PATH,temporalResolution))
    icDischarge = ee.ImageCollection("%s/global_historical_riverdischarge_%s_millionm3_5min_1960_2014" %(EE_PATH,temporalResolution))

    if temporalResolution == "year":
        imageWW = ee.Image(icWW.filter(ee.Filter.eq("year",year)).first())
        imageDischarge = ee.Image(icDischarge.filter(ee.Filter.eq("year",year)).first())
        image = imageWW.divide(imageDischarge)
    elif temporalResolution == "month":
        imageWW = ee.Image(icWW.filter(ee.Filter.eq("year",year)).filter(ee.Filter.eq("month",month)).first())
        imageDischarge = ee.Image(icDischarge.filter(ee.Filter.eq("year",year)).filter(ee.Filter.eq("month",month)).first())
        imageWW.divide(imageDischarge)
    else:
        image = -9999
        
    image = image.set(properties)
    description = "WS5min_%sY%0.4dM%0.2dV%0.2d" %(temporalResolution,year,month,OUTPUT_VERSION)
    assetID = "%s/global_historical_WS5min_%s_millionm3_5min_1960_2014/global_historical_WS5min_%s_millionm3_5min_1960_2014Y%0.4dM%0.2d" %(EE_PATH,temporalResolution,temporalResolution,year,month)

    task = ee.batch.Export.image.toAsset(
        image =  ee.Image(image),
        description = description,
        assetId = assetID,
        dimensions = dimensions,
        crs = CRS,
        crsTransform = crsTransform,
        maxPixels = MAXPIXELS     
    )
    task.start()

In [9]:
for demandType in demandTypes:
    for temporalResolution in temporalResolutions:
        createCollections(demandType,temporalResolution)

('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWW_year_millionm3_5min_1960_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWW_year_millionm3_5min_1960_2014 already exists\n')
('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWW_month_millionm3_5min_1960_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWW_month_millionm3_5min_1960_2014 already exists\n')
('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWN_year_millionm3_5min_1960_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWN_year_millionm3_5min_1960_2014 already exists\n')
('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWN_month_millionm3_5min_1960_2014', 'Asset projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_PTotWN_month_millionm3_5min_1960_2014 already exists\n')


In [10]:
startLoop = datetime.datetime.now()
for demandType in demandTypes:
    for temporalResolution in temporalResolutions:
        try:            
            if temporalResolution == "year":
                month = 12
                for year in range(YEARMIN,YEARMAX+1):
                    totalDemand(year,month,demandType,temporalResolution)
            elif temporalResolution == "month":
                for year in range(YEARMIN,YEARMAX+1): 
                    for month in range(1,13):
                        totalDemand(year,month,demandType,temporalResolution)
        except:
            print "error"

(1960, 12, 'WW', 'year')
0:00:00.000340
(1961, 12, 'WW', 'year')
0:00:01.726048
(1962, 12, 'WW', 'year')
0:00:03.472082
(1963, 12, 'WW', 'year')
0:00:05.163750
(1964, 12, 'WW', 'year')
0:00:06.928014
(1965, 12, 'WW', 'year')
0:00:08.667153
(1966, 12, 'WW', 'year')
0:00:10.312396
(1967, 12, 'WW', 'year')
0:00:12.073634
(1968, 12, 'WW', 'year')
0:00:13.752761
(1969, 12, 'WW', 'year')
0:00:15.498729
(1970, 12, 'WW', 'year')
0:00:17.135769
(1971, 12, 'WW', 'year')
0:00:18.818045
(1972, 12, 'WW', 'year')
0:00:20.503534
(1973, 12, 'WW', 'year')
0:00:22.107735
(1974, 12, 'WW', 'year')
0:00:23.812131
(1975, 12, 'WW', 'year')
0:00:25.573513
(1976, 12, 'WW', 'year')
0:00:27.272914
(1977, 12, 'WW', 'year')
0:00:28.980465
(1978, 12, 'WW', 'year')
0:00:30.730774
(1979, 12, 'WW', 'year')
0:00:32.436766
(1980, 12, 'WW', 'year')
0:00:34.106645
(1981, 12, 'WW', 'year')
0:00:35.864874
(1982, 12, 'WW', 'year')
0:00:37.557285
(1983, 12, 'WW', 'year')
0:00:39.189827
(1984, 12, 'WW', 'year')
0:00:39.924169


(1972, 6, 'WW', 'month')
0:05:44.928340
(1972, 7, 'WW', 'month')
0:05:46.731240
(1972, 8, 'WW', 'month')
0:05:48.572696
(1972, 9, 'WW', 'month')
0:05:49.969939
(1972, 10, 'WW', 'month')
0:05:51.848608
(1972, 11, 'WW', 'month')
0:05:53.697609
(1972, 12, 'WW', 'month')
0:05:55.594648
(1973, 1, 'WW', 'month')
0:05:57.421717
(1973, 2, 'WW', 'month')
0:05:59.260868
(1973, 3, 'WW', 'month')
0:06:01.136582
(1973, 4, 'WW', 'month')
0:06:02.902550
(1973, 5, 'WW', 'month')
0:06:04.709696
(1973, 6, 'WW', 'month')
0:06:06.595739
(1973, 7, 'WW', 'month')
0:06:08.427230
(1973, 8, 'WW', 'month')
0:06:10.281370
(1973, 9, 'WW', 'month')
0:06:12.114023
(1973, 10, 'WW', 'month')
0:06:14.006182
(1973, 11, 'WW', 'month')
0:06:15.957480
(1973, 12, 'WW', 'month')
0:06:17.783057
(1974, 1, 'WW', 'month')
0:06:19.864963
(1974, 2, 'WW', 'month')
0:06:21.723856
(1974, 3, 'WW', 'month')
0:06:23.519951
(1974, 4, 'WW', 'month')
0:06:25.359129
(1974, 5, 'WW', 'month')
0:06:27.212263
(1974, 6, 'WW', 'month')
0:06:29.0

(1989, 6, 'WW', 'month')
0:11:34.701159
(1989, 7, 'WW', 'month')
0:11:36.398635
(1989, 8, 'WW', 'month')
0:11:37.557965
(1989, 9, 'WW', 'month')
0:11:39.203627
(1989, 10, 'WW', 'month')
0:11:40.872290
(1989, 11, 'WW', 'month')
0:11:42.564677
(1989, 12, 'WW', 'month')
0:11:44.242395
(1990, 1, 'WW', 'month')
0:11:45.932939
(1990, 2, 'WW', 'month')
0:11:47.579226
(1990, 3, 'WW', 'month')
0:11:49.202511
(1990, 4, 'WW', 'month')
0:11:50.819152
(1990, 5, 'WW', 'month')
0:11:52.577939
(1990, 6, 'WW', 'month')
0:11:54.234999
(1990, 7, 'WW', 'month')
0:11:55.894717
(1990, 8, 'WW', 'month')
0:11:57.110743
(1990, 9, 'WW', 'month')
0:11:58.758877
(1990, 10, 'WW', 'month')
0:12:00.417167
(1990, 11, 'WW', 'month')
0:12:02.107355
(1990, 12, 'WW', 'month')
0:12:03.809191
(1991, 1, 'WW', 'month')
0:12:05.592430
(1991, 2, 'WW', 'month')
0:12:07.283631
(1991, 3, 'WW', 'month')
0:12:08.957706
(1991, 4, 'WW', 'month')
0:12:10.611443
(1991, 5, 'WW', 'month')
0:12:12.238426
(1991, 6, 'WW', 'month')
0:12:14.1

(2006, 6, 'WW', 'month')
0:17:28.004714
(2006, 7, 'WW', 'month')
0:17:29.891268
(2006, 8, 'WW', 'month')
0:17:31.751608
(2006, 9, 'WW', 'month')
0:17:33.625866
(2006, 10, 'WW', 'month')
0:17:35.523670
(2006, 11, 'WW', 'month')
0:17:37.359566
(2006, 12, 'WW', 'month')
0:17:39.310166
(2007, 1, 'WW', 'month')
0:17:41.167059
(2007, 2, 'WW', 'month')
0:17:42.512534
(2007, 3, 'WW', 'month')
0:17:44.310903
(2007, 4, 'WW', 'month')
0:17:46.117472
(2007, 5, 'WW', 'month')
0:17:47.992584
(2007, 6, 'WW', 'month')
0:17:49.828629
(2007, 7, 'WW', 'month')
0:17:51.643017
(2007, 8, 'WW', 'month')
0:17:53.472552
(2007, 9, 'WW', 'month')
0:17:55.341569
(2007, 10, 'WW', 'month')
0:17:57.147614
(2007, 11, 'WW', 'month')
0:17:58.493291
(2007, 12, 'WW', 'month')
0:17:59.883369
(2008, 1, 'WW', 'month')
0:18:01.777074
(2008, 2, 'WW', 'month')
0:18:03.540138
(2008, 3, 'WW', 'month')
0:18:05.468769
(2008, 4, 'WW', 'month')
0:18:07.275073
(2008, 5, 'WW', 'month')
0:18:09.142850
(2008, 6, 'WW', 'month')
0:18:10.9

(1963, 11, 'WN', 'month')
0:23:18.167279
(1963, 12, 'WN', 'month')
0:23:19.838180
(1964, 1, 'WN', 'month')
0:23:21.532792
(1964, 2, 'WN', 'month')
0:23:23.306589
(1964, 3, 'WN', 'month')
0:23:25.145819
(1964, 4, 'WN', 'month')
0:23:26.840228
(1964, 5, 'WN', 'month')
0:23:28.549097
(1964, 6, 'WN', 'month')
0:23:30.253064
(1964, 7, 'WN', 'month')
0:23:31.945417
(1964, 8, 'WN', 'month')
0:23:33.598836
(1964, 9, 'WN', 'month')
0:23:35.354785
(1964, 10, 'WN', 'month')
0:23:37.087271
(1964, 11, 'WN', 'month')
0:23:38.840563
(1964, 12, 'WN', 'month')
0:23:40.799830
(1965, 1, 'WN', 'month')
0:23:42.573263
(1965, 2, 'WN', 'month')
0:23:44.295237
(1965, 3, 'WN', 'month')
0:23:45.502194
(1965, 4, 'WN', 'month')
0:23:47.175088
(1965, 5, 'WN', 'month')
0:23:48.884314
(1965, 6, 'WN', 'month')
0:23:50.770821
(1965, 7, 'WN', 'month')
0:23:51.968589
(1965, 8, 'WN', 'month')
0:23:53.730369
(1965, 9, 'WN', 'month')
0:23:55.426153
(1965, 10, 'WN', 'month')
0:23:57.170455
(1965, 11, 'WN', 'month')
0:23:58.

(1980, 11, 'WN', 'month')
0:29:10.732282
(1980, 12, 'WN', 'month')
0:29:12.459163
(1981, 1, 'WN', 'month')
0:29:14.205142
(1981, 2, 'WN', 'month')
0:29:15.913030
(1981, 3, 'WN', 'month')
0:29:17.604349
(1981, 4, 'WN', 'month')
0:29:19.359930
(1981, 5, 'WN', 'month')
0:29:21.047403
(1981, 6, 'WN', 'month')
0:29:22.759432
(1981, 7, 'WN', 'month')
0:29:24.458605
(1981, 8, 'WN', 'month')
0:29:26.244622
(1981, 9, 'WN', 'month')
0:29:28.027495
(1981, 10, 'WN', 'month')
0:29:29.709113
(1981, 11, 'WN', 'month')
0:29:31.369726
(1981, 12, 'WN', 'month')
0:29:33.160270
(1982, 1, 'WN', 'month')
0:29:34.851976
(1982, 2, 'WN', 'month')
0:29:36.489958
(1982, 3, 'WN', 'month')
0:29:38.200837
(1982, 4, 'WN', 'month')
0:29:39.929847
(1982, 5, 'WN', 'month')
0:29:41.634345
(1982, 6, 'WN', 'month')
0:29:43.315458
(1982, 7, 'WN', 'month')
0:29:45.007771
(1982, 8, 'WN', 'month')
0:29:46.749918
(1982, 9, 'WN', 'month')
0:29:48.461569
(1982, 10, 'WN', 'month')
0:29:52.274232
(1982, 11, 'WN', 'month')
0:29:53.

(1997, 11, 'WN', 'month')
0:34:55.829438
(1997, 12, 'WN', 'month')
0:34:57.519524
(1998, 1, 'WN', 'month')
0:34:59.147170
(1998, 2, 'WN', 'month')
0:35:00.826053
(1998, 3, 'WN', 'month')
0:35:02.541457
(1998, 4, 'WN', 'month')
0:35:04.349184
(1998, 5, 'WN', 'month')
0:35:06.070319
(1998, 6, 'WN', 'month')
0:35:07.677547
(1998, 7, 'WN', 'month')
0:35:09.393356
(1998, 8, 'WN', 'month')
0:35:11.058996
(1998, 9, 'WN', 'month')
0:35:12.741257
(1998, 10, 'WN', 'month')
0:35:14.418844
(1998, 11, 'WN', 'month')
0:35:16.043392
(1998, 12, 'WN', 'month')
0:35:17.733378
(1999, 1, 'WN', 'month')
0:35:19.403819
(1999, 2, 'WN', 'month')
0:35:21.104197
(1999, 3, 'WN', 'month')
0:35:22.773240
(1999, 4, 'WN', 'month')
0:35:24.448610
(1999, 5, 'WN', 'month')
0:35:26.118555
(1999, 6, 'WN', 'month')
0:35:27.829131
(1999, 7, 'WN', 'month')
0:35:29.488242
(1999, 8, 'WN', 'month')
0:35:31.191206
(1999, 9, 'WN', 'month')
0:35:32.873582
(1999, 10, 'WN', 'month')
0:35:34.530661
(1999, 11, 'WN', 'month')
0:35:36.

(2014, 11, 'WN', 'month')
0:40:55.272572
(2014, 12, 'WN', 'month')
0:40:57.078742


In [14]:
for temporalResolution in temporalResolutions:
    createCollectionsWS(temporalResolution)

('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_WS5min_year_millionm3_5min_1960_2014', '')
('earthengine create collection projects/WRI-Aquaduct/PCRGlobWB20V07/global_historical_WS5min_month_millionm3_5min_1960_2014', '')


## RUN THIS CELL! 29/11/2017

In [19]:
startLoop = datetime.datetime.now()

for temporalResolution in temporalResolutions:
    try:            
        if temporalResolution == "year":
            month = 12
            for year in range(YEARMIN,YEARMAX+1):
                waterStressUncapped(year,month,temporalResolution)
        elif temporalResolution == "month":
            for year in range(YEARMIN,YEARMAX+1): 
                for month in range(1,13):
                    waterStressUncapped(year,month,temporalResolution)
    except:
        print "error"

(1960, 12, 'year')
0:00:00.000306
(1961, 12, 'year')
0:00:00.909091
(1962, 12, 'year')
0:00:01.532437
(1963, 12, 'year')
0:00:02.164679
(1964, 12, 'year')
0:00:02.789690
(1965, 12, 'year')
0:00:03.405375
(1966, 12, 'year')
0:00:04.033705
(1967, 12, 'year')
0:00:04.667801
(1968, 12, 'year')
0:00:05.281317
(1969, 12, 'year')
0:00:05.894182
(1970, 12, 'year')
0:00:06.523920
(1971, 12, 'year')
0:00:07.140369
(1972, 12, 'year')
0:00:07.798933
(1973, 12, 'year')
0:00:08.412115
(1974, 12, 'year')
0:00:09.029297
(1975, 12, 'year')
0:00:09.643240
(1976, 12, 'year')
0:00:09.804179
(1977, 12, 'year')
0:00:10.429292
(1978, 12, 'year')
0:00:11.045257
(1979, 12, 'year')
0:00:11.666331
(1980, 12, 'year')
0:00:11.819429
(1981, 12, 'year')
0:00:12.435441
(1982, 12, 'year')
0:00:13.070313
(1983, 12, 'year')
0:00:13.711442
(1984, 12, 'year')
0:00:14.349852
(1985, 12, 'year')
0:00:14.973090
(1986, 12, 'year')
0:00:15.609741
(1987, 12, 'year')
0:00:16.226043
(1988, 12, 'year')
0:00:16.856356
(1989, 12, 'ye

In [11]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:41:08.222200
